In [1]:
%%bash
set -e

REPO_URL="https://github.com/milo1409/BigDataPryFinal.git"
REPO_DIR="/content/BigDataPryFinal"
BRANCH="main"   # cambia a "master" si tu repo usa master

cd /content

if [ ! -d "$REPO_DIR/.git" ]; then
  rm -rf "$REPO_DIR"
  git clone --branch "$BRANCH" "$REPO_URL" "$REPO_DIR"
else
  cd "$REPO_DIR"
  git fetch origin "$BRANCH"
  git reset --hard "origin/$BRANCH"
  git clean -fd
fi

echo "Repo listo en: $REPO_DIR"
git -C "$REPO_DIR" rev-parse --short HEAD

HEAD is now at 928ac7c ajustes
Removing data/dashboard/
Removing data/procesada/
Removing src/__pycache__/
Repo listo en: /content/BigDataPryFinal
928ac7c


From https://github.com/milo1409/BigDataPryFinal
 * branch            main       -> FETCH_HEAD
   909069d..928ac7c  main       -> origin/main


In [2]:
import sys
import importlib
sys.path.append('/content/BigDataPryFinal/src')

import utilities
importlib.reload(utilities)
from utilities import Utils

json_config = Utils.read_json_config('/content/BigDataPryFinal/config/configuracion.json')
Utils.pip_install_requirements_from_config(json_config)


0

In [3]:
import utilities
importlib.reload(utilities)
from utilities import Utils

import procesar_datos
importlib.reload(procesar_datos)
from procesar_datos import LocalidadesJoinSpark

import conectar_datos
importlib.reload(conectar_datos)
from conectar_datos import ExtraerDatosProcesamiento

import cargar_datos
importlib.reload(cargar_datos)
from cargar_datos import DataLoader123

import deplegar_dashboard
importlib.reload(deplegar_dashboard)
from deplegar_dashboard import DashboardIncidentesSpark

In [ ]:
# Procesar los 36 meses solicitados
df_total, df_incons = DataLoader123.procesar_todos_csv_crudos(json_config)

# Estandarizar todos los dataframes
df_total = DataLoader123.estandarizacion_columnas(df_total)

# Inicializar spark para generar las particiones y analisis
spark = Utils.get_spark(app_name="BigDataPryFinal")

# Procesar los datos
datos = ExtraerDatosProcesamiento(spark, Utils, json_config)

# Generar el formato parquet para las particiones y el general
parquet = datos.generar_parquets_dashboard_spark(df_total,mode="overwrite", limpiar_procesada=True, limpiar_dashboard=True )

# Usar broadcast para geolocalizar las localidades
df_incidentes = spark.read.parquet("/content/BigDataPryFinal/data/procesada/incidentes")
joiner = LocalidadesJoinSpark(spark, Utils, json_config)
df_agg = joiner.construir_df_agg_mapa(
    df_incidentes=df_incidentes,
    geojson_filename="localidades_xy.geojson"
)

# Inicializar el dashboard
json_config["ts_col"] = "FECHA_INICIO_DESPLAZAMIENTO_MOVIL"
dash_app = DashboardIncidentesSpark(spark, Utils, json_config)

In [ ]:
# Desplegar el dashboard
dash_app.run("inline", 900, debug=False)